In [18]:
import tensorflow as tf
from keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer
from keras.layers import Embedding, Input, GlobalAveragePooling1D, Dense, Flatten
from keras.datasets import imdb
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [19]:
def transformer_block(embed_dim, num_heads, ff_dim, dropout_rate):
    att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
    ffn = Sequential(
        [Dense(ff_dim, activation="relu"),
         Dense(embed_dim), ]
    )
    layernorm1 = LayerNormalization(epsilon=1e-6)
    layernorm2 = LayerNormalization(epsilon=1e-6)
    dropout1 = Dropout(dropout_rate)
    dropout2 = Dropout(dropout_rate)
    
    def call(inputs, training):
        attn_output = att(inputs, inputs)
        attn_output = dropout1(attn_output, training=training)
        out1 = layernorm1(inputs + attn_output)
        ffn_output = ffn(out1)
        ffn_output = dropout2(ffn_output, training=training)
        return layernorm2(out1 + ffn_output)
    
    return call

In [20]:
#Load training data
train_df = pd.read_csv('../data/cleaned_train.csv')

# Extract the columns you want to use as input features
columns = ['BERT_sentiment_score', 'normalised_word_count', 'pub_day', 'pub_hour']
# Extract input features from the dataframe
x_train = np.array(train_df[columns].dropna())

y_train = np.array(train_df['n_comments'].dropna())

In [21]:
#Load test data
test_df = pd.read_csv('../data/cleaned_test.csv')

x_test = np.array(test_df[columns].dropna())

In [22]:
def label_encode(raw_data):
    # Concatenate train and test data vertically to ensure consistent label encoding
    combined_df = pd.concat([train_df['topic'], test_df['topic']], axis=0)

    # Create an instance of LabelEncoder
    le = LabelEncoder()

    # Fit and transform the combined data using LabelEncoder
    combined_encoded = le.fit_transform(combined_df)
    
    return np.array(le.transform(raw_data.dropna()))

In [23]:
# Apply the LabelEncoder transformation on train data
train_topic_encoded = pd.DataFrame(label_encode(train_df['topic']), columns=['topic encoded'])
# Concatenate the 'topic_encoded' tensor with the other input features
x_train = np.concatenate((x_train, train_topic_encoded), axis=1)

In [24]:
# Apply the LabelEncoder transformation on train data
test_topic_encoded = pd.DataFrame(label_encode(test_df['topic']), columns=['topic encoded'])
# Concatenate the 'topic_encoded' tensor with the other input features
x_test = np.concatenate([x_test, test_topic_encoded], axis=1)

In [25]:
# Training and validation split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [26]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_mae', 
                               # Number of epochs to wait for improvement
                               patience=10,  
                               verbose=1, 
                               # Restore the weights of the best epoch
                               restore_best_weights=True)  

In [27]:
# Define model architecture
max_seq_length = x_train.shape[1]
vocab_size = int(np.max(x_train) + 1)
embedding_dim = 32
num_heads = 2
ff_dim = 32
dropout_rate = 0.1

inputs = Input(shape=(max_seq_length,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_seq_length)(inputs)
transformer_block_fn = transformer_block(embed_dim=embedding_dim, 
                                         num_heads=num_heads, ff_dim=ff_dim, 
                                         dropout_rate=dropout_rate)
transformer_block = transformer_block_fn(embedding_layer, training=True)
pooling_layer = GlobalAveragePooling1D()(transformer_block)
dropout_layer = Dropout(rate=0.1)(pooling_layer)
outputs = Dense(units=1, activation='linear')(dropout_layer)  # Change activation to 'linear' for regression

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer=Adam(), loss='mse', metrics=['mae'])  # Change loss function to 'mse' for regression

# Train the model
model.fit(x_train, y_train, batch_size=64, epochs=100, validation_data=(x_val, y_val), 
          callbacks=[early_stopping])

Epoch 1/100
156/156 [==============================] - 6s 15ms/step - loss: 363435.2812 - mae: 297.4564 - val_loss: 368373.4062 - val_mae: 290.3582
Epoch 2/100
156/156 [==============================] - 2s 11ms/step - loss: 356260.2812 - mae: 290.5601 - val_loss: 360009.4688 - val_mae: 283.6694
Epoch 3/100
156/156 [==============================] - 2s 10ms/step - loss: 346226.3750 - mae: 284.0396 - val_loss: 348980.4688 - val_mae: 277.6183
Epoch 4/100
156/156 [==============================] - 2s 11ms/step - loss: 334469.4688 - mae: 279.2980 - val_loss: 336867.5938 - val_mae: 274.1417
Epoch 5/100
156/156 [==============================] - 2s 11ms/step - loss: 321926.5625 - mae: 276.6255 - val_loss: 324455.4062 - val_mae: 272.3585
Epoch 6/100
156/156 [==============================] - 2s 11ms/step - loss: 309195.0000 - mae: 272.3262 - val_loss: 311921.0312 - val_mae: 265.7162
Epoch 7/100
156/156 [==============================] - 2s 10ms/step - loss: 296563.5000 - mae: 269.0978 - val_lo

In [75]:
# save the trained model
model.save('../models/transformer_regression_model.h5')

In [29]:
# Evaluate the model on validation data
mse, mae = model.evaluate(x_val, y_val)
print('Validation MSE:', mse)
print('Validation MAE:', mae)

78/78 [==============================] - 0s 4ms/step - loss: 211007.6719 - mae: 240.7832
Validation MSE: 211007.671875
Validation MAE: 240.78318786621094
